In [ ]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import spatial
from datetime import timedelta
import seaborn as sns
from sklearn import metrics
from sklearn.neural_network import MLPRegressor


#error definition
def mean_absolute_percentage_error_value(y_true, y_pred):
        to_mean = []
        for true, pred in zip(y_true, y_pred):
            if true != 0:
                to_mean.append(np.abs((true - pred) / true))
        return np.mean(to_mean) * 100

def mae( y_true, y_pred):
        return metrics.mean_absolute_error(y_true, y_pred)

def rmse( y_true, y_pred):
        return np.sqrt(metrics.mean_squared_error(y_true, y_pred))


    
# automate input of files
data_resol = ['1h', '30min', '15min']
pred_horizons = [24, 24, 24]


for res, hor in zip(data_resol, pred_horizons): 
    print('************************** data resolution [{}] ******************************************'.format(res))
    
    df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_{}_no_outliers.csv'.format(res), index_col=0, parse_dates=[0])


    # data preparation
    df.power = df.power.interpolate(method='linear', limit = 1 )
    df.speed = df.speed.interpolate(method='linear', limit = 1 )
    df = df.dropna(subset=['speed', 'power'], how='any')

    max_power =np.max(df.power)

    # parameters of prediction
    time_horizon_lengths = np.arange(1,hor+1,1)
    df_f_indexes = np.arange(1,3000,150)

    
    list_nMAE_for_horizon =[]
    list_nRMSE_for_horizon = []


    for t in time_horizon_lengths:

        print('time horizon {}'.format(t), '\n')

        list_nMAE = []
        list_nRMSE = []

        for i in df_f_indexes:
            
            # define test data
            X_test = df.speed.iloc[i:i+t].values     # iloc[0:1] == [1]
            actual_results = df.power.iloc[i:i+t].values

            # define training data
            a = df.speed[:i].values
            b = df.speed[i+t:].values
            X_train =  np.concatenate([a,b])

            c = df.power[:i].values
            d = df.power[i+t:].values
            y_train = np.concatenate([c,d])

            # prepare the input in the right structure
            X_train = X_train.reshape(len(X_train), 1)
            X_test = X_test.reshape(len(X_test), 1)

            # model definition and parameters
            svr_rbf = MLPRegressor(hidden_layer_sizes=(100, ), 
                                    activation='logistic', 
                                    learning_rate='adaptive', 
                                    max_iter=2000, 

                                    solver='adam', 
                                    alpha=0.0001, 
                                    batch_size='auto', 
                                    learning_rate_init=0.001, 
                                    power_t=0.5, 
                                    shuffle=True, 
                                    random_state=None, 
                                    tol=0.0001, verbose=False, 
                                    warm_start=False, 
                                    momentum=0.9, 
                                    nesterovs_momentum=True, 
                                    early_stopping=False, 
                                    validation_fraction=0.1, 
                                    beta_1=0.9, 
                                    beta_2=0.999, 
                                    epsilon=1e-08)

            y_rbf = svr_rbf.fit(X_train, y_train).predict(X_test)

            # calculate errors
            MAE_1h = mae(actual_results, y_rbf)
            RMSE_1h = rmse(actual_results, y_rbf)

            # normalized errors
            nMAE_1h = (MAE_1h/max_power)*100
            nRMSE_1h = (RMSE_1h/max_power)*100

            # list of errors for a horizon
            list_nMAE.append(nMAE_1h)
            list_nRMSE.append(nRMSE_1h)

        # avg error for a horizon
        avg_nMAE = sum(list_nMAE)/(len(list_nMAE))  
        avg_nRMSE = sum(list_nRMSE)/(len(list_nRMSE))

        # list of errors for all horizons
        list_nMAE_for_horizon.append(avg_nMAE)
        list_nRMSE_for_horizon.append(avg_nRMSE)

        print('list_nMAE_for_horizon: ', list_nMAE_for_horizon, '\n')
        print('list_nRMSE_for_horizon: ', list_nRMSE_for_horizon, '\n')

    # save results to columns
    csv_input = pd.read_csv('ANN_results_all_data.csv')
    csv_input['nMAE_{}'.format(res)] = list_nMAE_for_horizon
    csv_input['nRMSE_{}'.format(res)] = list_nRMSE_for_horizon
    csv_input.to_csv('ANN_results_all_data.csv', index=False)

************************** data resolution [1h] ******************************************
time horizon 1 

list_nMAE_for_horizon:  [0.64469464739584936] 

list_nRMSE_for_horizon:  [0.64469464739584936] 

time horizon 2 

list_nMAE_for_horizon:  [0.64469464739584936, 0.83147089436348742] 

list_nRMSE_for_horizon:  [0.64469464739584936, 0.91847987910185913] 

time horizon 3 

list_nMAE_for_horizon:  [0.64469464739584936, 0.83147089436348742, 0.78787489752204354] 

list_nRMSE_for_horizon:  [0.64469464739584936, 0.91847987910185913, 0.88718985776739956] 

time horizon 4 

list_nMAE_for_horizon:  [0.64469464739584936, 0.83147089436348742, 0.78787489752204354, 0.79686269864417203] 

list_nRMSE_for_horizon:  [0.64469464739584936, 0.91847987910185913, 0.88718985776739956, 0.94543484874308026] 

time horizon 5 

list_nMAE_for_horizon:  [0.64469464739584936, 0.83147089436348742, 0.78787489752204354, 0.79686269864417203, 0.84760197688827821] 

list_nRMSE_for_horizon:  [0.64469464739584936, 0.918

C:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


list_nMAE_for_horizon:  [0.64469464739584936, 0.83147089436348742, 0.78787489752204354, 0.79686269864417203, 0.84760197688827821, 0.83521437009663746, 0.87837774078697461, 0.94322068643118906, 1.0028457994294422, 1.0584049313312061, 1.0987558967067277, 1.1828485896998042, 1.1919106067333365, 1.2316815040981901, 1.2442044565186914, 1.2896489363391426, 1.2646179898954435, 1.2598128028353095, 1.2512376124261628, 1.2773809546186348, 1.2602476675143641] 

list_nRMSE_for_horizon:  [0.64469464739584936, 0.91847987910185913, 0.88718985776739956, 0.94543484874308026, 1.0142289874241015, 1.002850205402591, 1.0545155751197739, 1.1312869388359217, 1.2194441522381871, 1.3003640045935598, 1.37569331915225, 1.4893271706157305, 1.5036258594056213, 1.6005235643081765, 1.6112161264031895, 1.6626874583682292, 1.6243162214692251, 1.6169606348290373, 1.6146207928481382, 1.6613481037906699, 1.6399406651928163] 

time horizon 22 

list_nMAE_for_horizon:  [0.64469464739584936, 0.83147089436348742, 0.787874897